In [12]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [13]:
# -------------------------------
# Boids core
# -------------------------------
class Boids:
    def __init__(
        self,
        n=100,
        width=10.0,
        height=6.0,
        perception=0.75,
        separation_radius=0.35,
        max_speed=2.0,
        max_force=0.05,
        w_align=1.0,
        w_cohere=0.8,
        w_separate=1.5,
        seed=None,
    ):
        self.n = n
        self.W = width
        self.H = height
        self.perception = perception
        self.sep_r = separation_radius
        self.max_speed = max_speed
        self.max_force = max_force
        self.w_align = w_align
        self.w_cohere = w_cohere
        self.w_separate = w_separate

        rng = np.random.default_rng(seed)
        # 初始化位置
        self.pos = np.column_stack([
            rng.uniform(0, self.W, size=n),
            rng.uniform(0, self.H, size=n),
        ])
        # 初始化速度
        headings = rng.uniform(0, 2*np.pi, size=n)
        speeds = rng.uniform(0.3*max_speed, 0.8*max_speed, size=n).reshape(-1, 1)
        self.vel = np.column_stack([np.cos(headings), np.sin(headings)]) * speeds
        self.acc = np.zeros_like(self.vel)

    def _wrap(self):
        """边界处理：环绕（torus）"""
        self.pos[:, 0] = np.mod(self.pos[:, 0], self.W)
        self.pos[:, 1] = np.mod(self.pos[:, 1], self.H)

    def _steer_to(self, desired, i):
        """计算转向力（限制最大加速度）"""
        steer = desired - self.vel[i]
        norm = np.linalg.norm(steer)
        if norm > self.max_force:
            steer = steer * (self.max_force / (norm + 1e-12))
        return steer

    def _neighbor_deltas(self, i):
        """计算个体 i 到所有邻居的相对位移（考虑环绕边界）"""
        dx = self.pos[:, 0] - self.pos[i, 0]
        dy = self.pos[:, 1] - self.pos[i, 1]
        dx = dx - np.round(dx / self.W) * self.W
        dy = dy - np.round(dy / self.H) * self.H
        return np.column_stack([dx, dy])

    def step(self, dt=0.05):
        """单步更新"""
        self.acc[:] = 0.0
        for i in range(self.n):
            delta = self._neighbor_deltas(i)
            dist = np.linalg.norm(delta, axis=1)
            mask = (dist > 0) & (dist < self.perception)

            if not np.any(mask):
                continue

            # ---- Alignment 对齐 ----
            avg_vel = np.mean(self.vel[mask], axis=0)
            steer_align = np.zeros(2)
            if np.linalg.norm(avg_vel) > 1e-9:
                desired = avg_vel / np.linalg.norm(avg_vel) * self.max_speed
                steer_align = self._steer_to(desired, i)

            # ---- Cohesion 聚合 ----
            center = self.pos[i] + np.mean(delta[mask], axis=0)
            steer_cohere = np.zeros(2)
            vec = center - self.pos[i]
            if np.linalg.norm(vec) > 1e-9:
                desired = vec / np.linalg.norm(vec) * self.max_speed
                steer_cohere = self._steer_to(desired, i)

            # ---- Separation 分离 ----
            close_mask = (dist > 0) & (dist < self.sep_r)
            steer_sep = np.zeros(2)
            if np.any(close_mask):
                away = -np.sum(delta[close_mask] / (dist[close_mask, None]**2 + 1e-12), axis=0)
                if np.linalg.norm(away) > 1e-9:
                    desired = away / np.linalg.norm(away) * self.max_speed
                    steer_sep = self._steer_to(desired, i)

            # 合并三种行为
            acc_i = (
                self.w_align * steer_align
                + self.w_cohere * steer_cohere
                + self.w_separate * steer_sep
            )
            norm = np.linalg.norm(acc_i)
            if norm > self.max_force:
                acc_i = acc_i * (self.max_force / (norm + 1e-12))
            self.acc[i] += acc_i

        # 更新速度和位置
        self.vel += self.acc
        speeds = np.linalg.norm(self.vel, axis=1, keepdims=True) + 1e-12
        too_fast = speeds > self.max_speed
        self.vel[too_fast[:, 0]] *= (self.max_speed / speeds[too_fast[:, 0]])
        min_speed = 0.15 * self.max_speed
        too_slow = speeds < min_speed
        self.vel[too_slow[:, 0]] *= (min_speed / speeds[too_slow[:, 0]])
        self.pos += self.vel * dt
        self._wrap()


In [14]:
# -------------------------------
# Visualization for Jupyter
# -------------------------------
def run_boids(n=150, width=10, height=6, fps=30, stepsize=0.05, seed=None):
    sim = Boids(n=n, width=width, height=height, seed=seed)

    fig, ax = plt.subplots(figsize=(width, height))
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.set_aspect('equal')
    ax.axis("off")

    scat = ax.scatter(sim.pos[:, 0], sim.pos[:, 1], s=20)

    def update(_):
        sim.step(dt=stepsize)
        scat.set_offsets(sim.pos)
        return scat,

    anim = FuncAnimation(fig, update, blit=True, interval=1000/fps)
    plt.close(fig)  # 防止重复显示
    return anim


In [15]:
anim = run_boids(n=200, width=12, height=8, fps=60)
from IPython.display import HTML
HTML(anim.to_jshtml())


C:\Users\Saru\AppData\Local\Temp\ipykernel_13364\1280986075.py:20: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  anim = FuncAnimation(fig, update, blit=True, interval=1000/fps)
Animation size has reached 20982159 bytes, exceeding the limit of 20971520.0. If you're sure you want a larger animation embedded, set the animation.embed_limit rc parameter to a larger value (in MB). This and further frames will be dropped.


KeyboardInterrupt: 